In [12]:
# %%
# Import necessary libraries for data handling, model utilization, and visualization
import numpy as np
import pandas as pd
import yfinance as yf  # For collecting financial data
import matplotlib.pyplot as plt
from typing import List, Dict, Optional, Tuple
from scipy.optimize import minimize 
from datetime import datetime, timedelta
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from collections import deque
# Import the custom Model class
from Model import Model

# Set the random seed for reproducibility across numpy and tensorflow
np.random.seed(123)
tf.random.set_seed(123)

plt.style.use('seaborn-darkgrid')

# Define the tickers and date range with consideration of trading days
TICKERS = ['AGG','DBC','VTI','^VIX']

# Approximate number of trading days per year (useful for annualizing returns)
TRADING_DAYS_PER_YEAR = 252

# Define transaction cost rate
C = 0.0001  # 0.01%

# Confirm setup
print("Setup complete: libraries imported, random seed set, and tickers defined.")

Setup complete: libraries imported, random seed set, and tickers defined.


C:\Users\wiztu\AppData\Local\Temp\ipykernel_16260\1252333898.py:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [13]:
# %%
# Data Collection Step
# Objective: Fetch historical adjusted close prices for defined tickers and date range

# Download data using yfinance for the specified tickers and date range
def get_data(tickers, start_date, end_date):
    """
    Retrieves historical adjusted close prices for the given tickers and date range.
    
    Parameters:
    - tickers: List of stock ticker symbols
    - start_date: Start date for historical data
    - end_date: End date for historical data
    
    Returns:
    - DataFrame of adjusted close prices, with each column representing a ticker
    """
    # Fetch data from yfinance
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    
    # Drop rows with missing values, if any, to ensure data continuity
    data.dropna(inplace=True)
    
    return data

# Fetch the data and display a quick preview
data = get_data(TICKERS, '2006-01-01', '2020-04-30')
print("Data fetched successfully. Sample data:")
print(data.head())

# Confirm data spans the expected range and has the expected number of columns
print(f"Data covers {len(data)} trading days with {len(data.columns)} assets.")

[*********************100%%**********************]  4 of 4 completed

Data fetched successfully. Sample data:
Ticker            AGG        DBC        VTI   ^VIX
Date                                              
2006-02-06  56.300163  20.889498  44.654270  13.04
2006-02-07  56.260765  20.285255  44.219501  13.59
2006-02-08  56.232719  20.198936  44.537613  12.83
2006-02-09  56.266445  20.388840  44.452793  13.12
2006-02-10  56.148479  20.017662  44.544689  12.87
Data covers 3582 trading days with 4 assets.


In [14]:
# %%
def preprocess_data(data, rolling_window=50):
    """
    Prepares data by calculating 50-day rolling averages and returns.

    Parameters:
    - data: DataFrame of historical adjusted close prices for assets
    - rolling_window: Window size for the rolling average

    Returns:
    - normalized_data: Smoothed prices, normalized to start at 1 for each asset
    - returns: Smoothed returns using a rolling mean of percentage changes
    """
    # Calculate rolling mean for prices and returns to smooth the data
    smoothed_prices = (data.rolling(window=rolling_window).mean()).dropna()
    smoothed_returns = (data.pct_change().rolling(window=rolling_window).mean()).dropna()
    # Normalize prices to start each asset's time series at 1
    normalized_data = smoothed_prices / smoothed_prices.iloc[0]
    
    return normalized_data, smoothed_returns


# Run preprocessing and display sample data
normalized_data, smoothed_returns = preprocess_data(data)
print("Data preprocessing complete. Sample normalized data:")
print(normalized_data.head())
print("\nSample daily returns:")
print(smoothed_returns.head())


Data preprocessing complete. Sample normalized data:
Ticker           AGG       DBC       VTI      ^VIX
Date                                              
2006-04-18  1.000000  1.000000  1.000000  1.000000
2006-04-19  0.999786  1.001443  1.000798  0.997130
2006-04-20  0.999555  1.003226  1.001815  0.993875
2006-04-21  0.999332  1.005552  1.002683  0.991806
2006-04-24  0.999172  1.007249  1.003528  0.989520

Sample daily returns:
Ticker           AGG       DBC       VTI      ^VIX
Date                                              
2006-04-19 -0.000212  0.001441  0.000813 -0.001952
2006-04-20 -0.000229  0.001788  0.001035 -0.002230
2006-04-21 -0.000221  0.002295  0.000882 -0.001198
2006-04-24 -0.000157  0.001709  0.000862 -0.001374
2006-04-25 -0.000182  0.002136  0.000757 -0.000993


In [15]:
# %%
class Portfolio:
    def __init__(self, initial_cash: float, assets: pd.DataFrame):
        """
        Initializes the Portfolio object.

        Parameters:
        - initial_cash: The starting value of the portfolio in cash.
        - assets: DataFrame of asset prices (historical data).
        """
        self.initial_cash = initial_cash
        self.current_value = initial_cash
        self.assets = assets  # Historical price data for the assets
        self.weights = np.zeros(len(assets.columns))  # Initialize weights as zero
        self.portfolio_history = []  # To track portfolio value over time
        self.rebalancing_dates = []  # To store rebalancing dates

    def rebalance(self, new_weights: np.array, target_volatility = None):
        """
        Rebalances the portfolio according to new weights.

        Parameters:
        - new_weights: Numpy array of asset allocations.
        """
        if len(new_weights) != len(self.assets.columns):
            raise ValueError("Number of weights must match the number of assets.")
        self.weights = new_weights
        if target_volatility is not None:
            self.apply_volatility_scaling(target_volatility=target_volatility,rolling_window = 50)

    def calculate_initial_shares(self, initial_cash, initial_prices):
        """
        Calculates the number of shares for each asset at the start of the testing period based on
        initial cash and allocation weights.

        Parameters:
        - initial_cash: The starting cash value of the portfolio.

        Returns:
        - shares: Dictionary with tickers as keys and the initial number of shares as values.
        """
        # Calculate the dollar amount allocated to each asset
        dollar_allocation = initial_cash * self.weights

        # Calculate the number of shares for each asset
        shares = (dollar_allocation // initial_prices).astype(int)  # Floor division to get whole shares

        # Return as a dictionary for easy readability
        return dict(zip(self.assets.columns, shares))
    
    def calculate_daily_returns(self):
        """
        Applies the current weights to asset returns and updates portfolio value over time.
        """
        # Calculate daily returns for each asset
        daily_returns = self.assets.pct_change().dropna()
        
        # Calculate portfolio returns by applying weights
        portfolio_returns = np.dot(daily_returns, self.weights)

        # Track the portfolio's value over time by compounding the returns
        for daily_ret in portfolio_returns:
            self.current_value *= (1 + daily_ret)
            self.portfolio_history.append(self.current_value)

    def update_portfolio_value(self, date):
        """
        Updates the portfolio value for a single date.
        """
        # Get the index of the date
        date_index = self.assets.index.get_loc(date)
        if date_index == 0:
            # First day, no previous day to compute return
            self.portfolio_history.append(self.current_value)
            return
        # Get the asset returns for that day
        previous_date = self.assets.index[date_index - 1]
        daily_return = self.assets.loc[date] / self.assets.loc[previous_date] - 1
        # Calculate portfolio return
        portfolio_return = np.dot(daily_return.values, self.weights)
        # Update portfolio value
        self.current_value *= (1 + portfolio_return)
        # Append to history
        self.portfolio_history.append(self.current_value)


    def track_portfolio_performance(self):
        """
        Tracks and prints the portfolio performance over time.
        """
        for date, value in zip(self.assets.index[1:], self.portfolio_history):
            print(f"Date: {date}, Portfolio Value: {value}")
    
    def reset(self):
        self.current_cash = self.initial_cash
        self.assets = self.initial_assets.copy()
        self.weights = np.zeros(len(self.assets.columns))  # Reset to no investments
        self.portfolio_history = []
        self.rebalancing_dates = []
        return self.assets.iloc[0].values

    def get_portfolio_value(self):
        """
        Returns the current value of the portfolio.
        """
        return self.current_value
    
    def plot_portfolio_value(self):
        """
        Plots the portfolio value over time.
        """
        plt.figure(figsize=(10, 6))
        plt.plot(self.assets.index[1:], self.portfolio_history, label="Portfolio Value")
        plt.title("Portfolio Value Over Time")
        plt.xlabel("Date")
        plt.ylabel("Portfolio Value")
        plt.legend()
        plt.show()


In [16]:
# %%
def calculate_metrics(portfolio_values):
    """
    Calculates performance metrics for the portfolio.

    Parameters:
    - portfolio_values: List of daily portfolio values over the testing period.

    Returns:
    - metrics: Dictionary containing various performance metrics.
    """
    # Convert portfolio values to daily returns
    portfolio_returns = np.diff(portfolio_values) / portfolio_values[:-1]
    
    # Number of days
    N = len(portfolio_returns)

    # Calculate Sharpe Ratio
    mean_return = np.mean(portfolio_returns)
    std_dev = np.std(portfolio_returns)
    sharpe_ratio = mean_return / std_dev * np.sqrt(TRADING_DAYS_PER_YEAR)
    
    # Calculate Sortino Ratio
    downside_returns = portfolio_returns[portfolio_returns < 0]
    downside_std_dev = np.std(downside_returns) if len(downside_returns) > 0 else 0
    sortino_ratio = mean_return / downside_std_dev * np.sqrt(TRADING_DAYS_PER_YEAR) if downside_std_dev != 0 else np.nan
    
    # Calculate Maximum Drawdown
    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (cumulative_max - portfolio_values) / cumulative_max
    max_drawdown = np.max(drawdowns)
    
    # Expected return (annualized)
    cumulative_return = (portfolio_values[-1] / portfolio_values[0]) - 1
    annualized_return = (1 + cumulative_return) ** (TRADING_DAYS_PER_YEAR / N) - 1

    # Standard deviation of returns (annualized)
    annualized_std = std_dev * np.sqrt(TRADING_DAYS_PER_YEAR)

    # Percentage of positive returns
    positive_returns = portfolio_returns[portfolio_returns > 0]
    percentage_positive = len(positive_returns) / len(portfolio_returns) * 100

    # Average profit / average loss (profit/loss ratio)
    average_profit = np.mean(portfolio_returns[portfolio_returns > 0]) if len(positive_returns) > 0 else 0
    average_loss = np.mean(portfolio_returns[portfolio_returns < 0]) if len(portfolio_returns[portfolio_returns < 0]) > 0 else 0
    profit_loss_ratio = (average_profit / -average_loss) if average_loss != 0 else np.nan

    metrics = {
        "Annualized Return": annualized_return,
        "Annualized Std Dev": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Max Drawdown": max_drawdown,
        "% Positive Returns": percentage_positive,
        "Profit/Loss Ratio": profit_loss_ratio
    }
    
    return metrics


In [ ]:
# %%
def equal_weighted_strategy(returns):
    """
    Creates an equal-weighted portfolio.

    Parameters:
    - returns: DataFrame of daily returns for each asset.

    Returns:
    - equal_weights: Numpy array of equal weights for each asset.
    """
    num_assets = returns.shape[1]
    equal_weights = np.ones(num_assets) / num_assets
    return equal_weights

def mean_variance_optimized_strategy(returns):
    """
    Creates a mean-variance optimized portfolio by maximizing the Sharpe Ratio.

    Parameters:
    - returns: DataFrame of daily returns for each asset.

    Returns:
    - optimized_weights: Numpy array of optimized weights for each asset.
    """
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    
    def neg_sharpe(weights):
        portfolio_return = np.dot(weights, mean_returns)
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -portfolio_return / portfolio_std if portfolio_std != 0 else 0

    # Constraints: Weights must sum to 1, and each weight must be between 0 and 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(returns.shape[1]))

    result = minimize(neg_sharpe, np.ones(returns.shape[1]) / returns.shape[1], bounds=bounds, constraints=constraints)
    optimized_weights = result.x
    
    return optimized_weights

def maximum_diversification(returns):
    """
    Perform maximum diversification optimization based on the given returns.

    Parameters:
    - returns: DataFrame of daily returns for each asset.

    Returns:
    - optimal_weights: Array of portfolio weights that maximize diversification.
    """
    # Calculate asset volatilities (standard deviation of each asset’s returns)
    asset_volatilities = returns.std()

    # Calculate the covariance matrix of returns
    cov_matrix = returns.cov()

    # Define the diversification ratio to be maximized
    def neg_diversification_ratio(weights):
        # Calculate the weighted average asset volatility
        weighted_volatility = np.dot(weights, asset_volatilities)
        
        # Calculate the portfolio volatility as the weighted covariance matrix
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        
        # Diversification ratio (we negate this because we want to maximize it)
        diversification_ratio = weighted_volatility / portfolio_volatility if portfolio_volatility != 0 else 0
        return -diversification_ratio  # Negate to turn this into a minimization problem

    # Constraints: weights sum to 1, and each weight between 0 and 1 (long-only portfolio)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(len(asset_volatilities)))

    # Initial guess (equal allocation)
    init_guess = np.ones(len(asset_volatilities)) / len(asset_volatilities)

    # Optimize to find weights that maximize diversification ratio
    result = minimize(neg_diversification_ratio, init_guess, bounds=bounds, constraints=constraints)
    optimal_weights = result.x
    
    return optimal_weights

In [18]:
def average_metrics(metrics_list):
    """
    Calculates the average of each metric in the list of metrics.
    """
    avg_metrics = {}
    keys = metrics_list[0].keys()
    for key in keys:
        avg_metrics[key] = np.mean([m[key] for m in metrics_list])
    return avg_metrics


In [19]:
# %%
# Define testing periods
training_end_dates = ['2010-12-31', '2012-12-31', '2014-12-31', '2016-12-31', '2018-12-31']
testing_start_dates = ['2011-01-01', '2013-01-01', '2015-01-01', '2017-01-01', '2019-01-01']
testing_end_dates = ['2012-12-31', '2014-12-31', '2016-12-31', '2018-12-31', '2020-04-30']

periods = list(zip(training_end_dates, testing_start_dates, testing_end_dates))

# Initialize lists to store performance metrics for each model
lstm_metrics = []
mvo_metrics = []
md_metrics = []

initial_cash = 100000

In [20]:
for period in periods:
    training_end, testing_start, testing_end = period
    print(f"\nProcessing period: Training up to {training_end}, Testing from {testing_start} to {testing_end}")

    # Get training data
    training_data = data.loc[:training_end].copy()
    # Get testing data
    testing_data = data.loc[testing_start:testing_end].copy()

    # Ensure we have enough data
    if len(testing_data) < 50:
        print("Not enough data for testing period")
        continue

    # LSTM Model
    lstm_model = Model()
    # Train the model and get allocation
    allocation = lstm_model.get_allocations(training_data)

    # Ensure allocation sums to 1
    allocation = allocation / np.sum(allocation)

    # Get testing returns
    testing_returns = testing_data.pct_change().dropna()
    # Align the returns with allocation
    testing_returns = testing_returns.loc[:, training_data.columns]

    # Calculate portfolio returns
    portfolio_returns = testing_returns.dot(allocation)

    # Calculate cumulative portfolio value
    portfolio_values = [initial_cash]
    for ret in portfolio_returns:
        new_value = portfolio_values[-1] * (1 + ret)
        portfolio_values.append(new_value)

    # Remove the first value (initial_cash)
    portfolio_values = portfolio_values[1:]

    # Calculate performance metrics
    metrics = calculate_metrics(portfolio_values)
    lstm_metrics.append(metrics)

    # MVO Strategy
    returns = testing_data.pct_change().dropna()
    # Ensure we have enough data
    if len(returns) < 50:
        print("Not enough data for MVO testing period")
        continue

    portfolio_values_mvo = [initial_cash]
    previous_weights_mvo = np.zeros(len(returns.columns))  # Initialize previous weights to zero

    for i in range(50, len(returns)):
        # Get past 50 days of returns
        past_returns = returns.iloc[i-50:i]

        # Calculate weights using mean_variance_optimized_strategy
        weights = mean_variance_optimized_strategy(past_returns)

        # Get next day's return
        next_return = returns.iloc[i]

        # Calculate transaction costs
        if i == 50:
            # First allocation, previous weights are zero
            transaction_cost = C * np.sum(np.abs(weights))
        else:
            transaction_cost = C * np.sum(np.abs(weights - previous_weights_mvo))
        
        # Deduct transaction costs from portfolio value
        portfolio_value_before_trade = portfolio_values_mvo[-1]
        transaction_cost_amount = transaction_cost * portfolio_value_before_trade
        portfolio_value_after_cost = portfolio_value_before_trade - transaction_cost_amount

        # Calculate portfolio return after transaction costs
        portfolio_return = np.dot(weights, next_return)

        # Update portfolio value
        new_value = portfolio_value_after_cost * (1 + portfolio_return)
        portfolio_values_mvo.append(new_value)

        # Update previous weights
        previous_weights_mvo = weights

    # Calculate performance metrics for MVO
    metrics_mvo = calculate_metrics(portfolio_values_mvo)
    mvo_metrics.append(metrics_mvo)

    # MD Strategy
    portfolio_values_md = [initial_cash]
    previous_weights_md = np.zeros(len(returns.columns))  # Initialize previous weights to zero

    for i in range(50, len(returns)):
        # Get past 50 days of returns
        past_returns = returns.iloc[i-50:i]

        # Calculate weights using maximum_diversification
        weights = maximum_diversification(past_returns)

        # Get next day's return
        next_return = returns.iloc[i]

        # Calculate transaction costs
        if i == 50:
            # First allocation, previous weights are zero
            transaction_cost = C * np.sum(np.abs(weights))
        else:
            transaction_cost = C * np.sum(np.abs(weights - previous_weights_md))
        
        # Deduct transaction costs from portfolio value
        portfolio_value_before_trade = portfolio_values_md[-1]
        transaction_cost_amount = transaction_cost * portfolio_value_before_trade
        portfolio_value_after_cost = portfolio_value_before_trade - transaction_cost_amount

        # Calculate portfolio return after transaction costs
        portfolio_return = np.dot(weights, next_return)

        # Update portfolio value
        new_value = portfolio_value_after_cost * (1 + portfolio_return)
        portfolio_values_md.append(new_value)

        # Update previous weights
        previous_weights_md = weights

    # Calculate performance metrics for MD
    metrics_md = calculate_metrics(portfolio_values_md)
    md_metrics.append(metrics_md)

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Processing period: Training up to 2010-12-31, Testing from 2011-01-01 to 2012-12-31
Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0130
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -0.0061    
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.0325
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.0786
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.0988
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.1051
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: -0.1152
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -0.1235
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -0.1233
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: -0.1303
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: -0.1264
Epoch 12/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.1332
Epoch 13/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: -0.1349
Epoch 14/100
19/19 ━━━━━

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: -0.0524
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.0938
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1061
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1140
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1263
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1419
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1378
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1508
Epoch 9/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1527
Epoch 10/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1602
Epoch 11/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1664
Epoch 12/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1635
Epoch 13/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1694
Epoch 14/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: -0.1710
Epoch 15/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - 

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: -0.0825
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1394
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1607
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1773
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1779
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1847
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1901
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1917
Epoch 9/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1935
Epoch 10/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1941
Epoch 11/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1964
Epoch 12/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1978
Epoch 13/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.2031
Epoch 14/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.2040
Epoch 15/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - 

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: -0.0603
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.0908
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1074
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1187
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1353
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1552
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1634
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1666
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1699
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1704
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1727
Epoch 12/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.1741
Epoch 13/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: -0.1788
Epoch 14/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1816
Epoch 15/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - 

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: -0.0294
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.0727
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1115
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1179
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1309
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1341
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1367
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1413
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1462
Epoch 10/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: -0.1508
Epoch 11/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1522
Epoch 12/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1565
Epoch 13/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1589
Epoch 14/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.1612
Epoch 15/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - 

In [21]:
# Calculate average metrics for each model
lstm_avg_metrics = average_metrics(lstm_metrics)
mvo_avg_metrics = average_metrics(mvo_metrics)
md_avg_metrics = average_metrics(md_metrics)

In [22]:
# Print the average metrics
print("\nAverage Metrics for LSTM Model:")
for key, value in lstm_avg_metrics.items():
    print(f"{key}: {value:.4f}")

print("\nAverage Metrics for MVO Strategy:")
for key, value in mvo_avg_metrics.items():
    print(f"{key}: {value:.4f}")

print("\nAverage Metrics for MD Strategy:")
for key, value in md_avg_metrics.items():
    print(f"{key}: {value:.4f}")


Average Metrics for LSTM Model:
Annualized Return: 0.1332
Annualized Std Dev: 0.0703
Sharpe Ratio: 1.8320
Sortino Ratio: 3.0171
Max Drawdown: 0.0590
% Positive Returns: 55.5006
Profit/Loss Ratio: 1.1544

Average Metrics for MVO Strategy:
Annualized Return: -0.0126
Annualized Std Dev: 0.1131
Sharpe Ratio: 0.0876
Sortino Ratio: 0.2279
Max Drawdown: 0.1612
% Positive Returns: 54.4243
Profit/Loss Ratio: 0.8456

Average Metrics for MD Strategy:
Annualized Return: 0.0882
Annualized Std Dev: 0.0582
Sharpe Ratio: 1.8101
Sortino Ratio: 2.6352
Max Drawdown: 0.0513
% Positive Returns: 57.6328
Profit/Loss Ratio: 1.0298
